In [152]:
from __future__ import print_function
from tqdm import tqdm

import torch
from torch.autograd import Variable
from torch import nn 

N, D_in, H, D_out = 64, 1000, 100, 10
lr = 1e-6
EPOCH = 500

In [175]:
# Random input , target and weights
x = Variable(torch.randn(N, D_in), requires_grad=False)
y = Variable(torch.randn(N, D_out), requires_grad=False)
w1, w2 = Variable(torch.randn(D_in, H), requires_grad=True), Variable(torch.randn(H, D_out), requires_grad=True)

In [116]:
# manual NN
for t in tqdm(range(EPOCH)):
    #forward passs
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    loss = (y_pred - y).pow(2).sum()
    if t > EPOCH * 0.96:
        print ("Epoch: {}, Squared Error: {}".format(t, loss.data[0]))
    #automatic backprop
    loss.backward()
    w1.data -= lr * w1.grad.data
    w2.data -= lr * w2.grad.data
    w1.grad.data.zero_()
    w2.grad.data.zero_()

100%|██████████| 200/200 [00:00<00:00, 577.58it/s]

Epoch: 193, Squared Error: 9.75530433655
Epoch: 194, Squared Error: 9.33067798615
Epoch: 195, Squared Error: 8.92428779602
Epoch: 196, Squared Error: 8.53660488129
Epoch: 197, Squared Error: 8.16599845886
Epoch: 198, Squared Error: 7.81162643433
Epoch: 199, Squared Error: 7.47278022766


In [153]:
# using nn module
model = nn.Sequential(nn.Linear(D_in, H),
                     nn.ReLU(),
                     nn.Linear(H, D_out))
loss_function = nn.MSELoss(size_average=False)

In [154]:
# Random input , target and weights
x = Variable(torch.randn(N, D_in), requires_grad=False)
y = Variable(torch.randn(N, D_out), requires_grad=False)
lr = 1e-4
for t in range(EPOCH):
    y_pred = model(x)
    loss = loss_function(y_pred, y)
    
    model.zero_grad()  # reset gradient
    loss.backward()  #backprop
    if t > EPOCH * 0.98:
        print ("Epoch: {}, Squared Error: {}".format(t, loss.data[0]))
    for p in model.parameters():
        p.data -= lr *p.grad.data
print("Yay!!, Done!")    

Epoch: 491, Squared Error: 7.18568435332e-06
Epoch: 492, Squared Error: 6.99095471646e-06
Epoch: 493, Squared Error: 6.80142784404e-06
Epoch: 494, Squared Error: 6.61750891595e-06
Epoch: 495, Squared Error: 6.43914290777e-06
Epoch: 496, Squared Error: 6.26421569905e-06
Epoch: 497, Squared Error: 6.09577773503e-06
Epoch: 498, Squared Error: 5.93052754994e-06
Epoch: 499, Squared Error: 5.77082073505e-06
Yay!!, Done!


In [185]:
# Random input and target data
x = Variable(torch.randn(N, D_in), requires_grad=False)
y = Variable(torch.randn(N, D_out), requires_grad=False)

In [190]:
# using ADAM optimizer
model = nn.Sequential(nn.Linear(D_in, H),
                     nn.ReLU(),
                     nn.Linear(H, D_out))
loss_fn = nn.MSELoss(size_average=False)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [191]:
# Let's run through all the EPOCH
for t in tqdm(range(EPOCH)):
    #feed forward
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    if t > EPOCH * 0.98:
        print ("Epoch: {}, Squared Error: {}".format(t, loss.data[0]))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

100%|██████████| 500/500 [00:01<00:00, 338.64it/s]

Epoch: 491, Squared Error: 8.06809021014e-06
Epoch: 492, Squared Error: 7.69934922573e-06
Epoch: 493, Squared Error: 7.34719742468e-06
Epoch: 494, Squared Error: 7.01082353771e-06
Epoch: 495, Squared Error: 6.6897182478e-06
Epoch: 496, Squared Error: 6.38247956886e-06
Epoch: 497, Squared Error: 6.08786695011e-06
Epoch: 498, Squared Error: 5.80704636377e-06
Epoch: 499, Squared Error: 5.53961581318e-06


In [194]:
# Lets create our on custom NN model 
class Runet(nn.Module):
    """ A custom Neural Net with 3 hidden Sequential layers"""
    
    def __init__(self,D_in, H1, H2, H3, D_out):
        super(Runet, self).__init__()
        self.linear1 = nn.Linear(D_in, H1)
        self.linear2 = nn.Linear(H1, H2)
        self.linear3 = nn.Linear(H2, H3)
        self.linear4 = nn.Linear(H3, D_out)
    
    def forward(self, x):
        """Feed foward part, where we describe how the NNet is
           connected and feed the input data
        """
        h1_relu = self.linear1(x).clamp(min=0)
        h2 = self.linear2(h1_relu)
        h3 = self.linear3(h2)
        y_pred = self.linear4(h3)
        return y_pred

In [282]:
# Dimensions and Hyperparameters of the Neural Net
N, D_in, H1, H2, H3, D_out = 64, 1000, 100, 100, 100, 10
lr = 1e-4
EPOCH=500

# Random Input and Target values
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)
X = Variable(x, requires_grad=False)
Y = Variable(y, requires_grad=False)

In [213]:
# create model
model = Runet(D_in, H1, H2, H3, D_out)
loss_fn = nn.MSELoss(size_average=False)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [214]:
# Lets run through all EPOCHS
for t in tqdm(range(EPOCH)):
    y_pred = model(X)
    loss = loss_fn(y_pred, Y)
    if t > EPOCH * 0.98 :
        print("EPOCH {} and LOSS {}".format(t, loss.data[0]))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
        
    
    

100%|██████████| 300/300 [00:01<00:00, 259.90it/s]

EPOCH 295 and LOSS 0.0123166963458
EPOCH 296 and LOSS 0.0134867755696
EPOCH 297 and LOSS 0.0123318349943
EPOCH 298 and LOSS 0.00936056207865
EPOCH 299 and LOSS 0.0104877706617


In [253]:
# Dynamic Model with weight sharing
import random

class DNet(nn.Module):
    """Dynamic Network, where there can be
    n - number of hidden layers."""
    
    def __init__(self, D_in, H, D_out):
        super(DNet, self).__init__()
        self.input = nn.Linear(D_in, H)
        self.hidden = nn.Linear(H, H)
        self.output = nn.Linear(H, D_out)
    
    def forward(self, X):
        h_relu = self.input(X).clamp(min=0)
        for h_layers in range(random.randint(1, 3)):
            h_relu = self.hidden(h_relu)
        h_relu = self.output(h_relu)
        return h_relu

In [283]:
# Dimensions and Hyperparameters of the Neural Net
N, D_in, H1, H2, H3, D_out = 64, 1000, 100, 100, 100, 10
lr = 1e-4
EPOCH=500

# Random Input and Target values
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)
X = Variable(x, requires_grad=False)
Y = Variable(y, requires_grad=False)

In [284]:
# create model
model = DNet(D_in, H1, D_out)
loss_fn = nn.MSELoss(size_average=False)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [286]:
for t in tqdm(range(EPOCH)):
    y_pred = model(X)
    loss = loss_fn(y_pred, Y)
    if t > EPOCH * 0.99:
        print("EPOCH: {} SQUARED ERROR: {}".format(t, loss.data[0]))
    optmizer.zero_grad()
    loss.backward()
    optimizer.step()
        
        
    

100%|██████████| 500/500 [00:01<00:00, 279.32it/s]

EPOCH: 496 SQUARED ERROR: 218.35949707
EPOCH: 497 SQUARED ERROR: 216.605056763
EPOCH: 498 SQUARED ERROR: 215.079040527
EPOCH: 499 SQUARED ERROR: 254.739318848


In [287]:
pwd

u'/home/rahul'

In [288]:
ll

total 68
drwxrwxr-x 20 rahul  4096 Jun 22 10:04 anaconda3/
drwxrwxr-x  5 rahul  4096 Jul 14 15:07 Coding/
drwxr-xr-x  3 rahul  4096 Jul 14 11:25 Desktop/
drwxr-xr-x  2 rahul  4096 Jun  5 16:21 Documents/
drwxr-xr-x  3 rahul  4096 Jul 12 09:55 Downloads/
drwxrwxr-x 10 rahul  4096 Jul  7 16:35 gym/
drwxrwxr-x  6 rahul  4096 Jun 29 11:12 machineLearning/
drwxr-xr-x  2 rahul  4096 Jun  5 16:21 Music/
drwxrwxr-x  2 rahul  4096 Jul 14 11:06 Notebooks/
drwxr-xr-x  2 rahul  4096 Jun  5 16:21 Pictures/
drwxr-xr-x  2 rahul  4096 Jun  5 16:21 Public/
-rw-rw-r--  1 rahul 12047 Jul 18 13:51 pytorch_basics.ipynb
drwxrwxr-x  2 rahul  4096 Jun  6 16:41 scikit_learn_data/
drwxr-xr-x  2 rahul  4096 Jun  5 16:21 Templates/
drwxr-xr-x  2 rahul  4096 Jun  5 16:21 Videos/


In [ ]:
scp